### Import library

In [1]:
import pandas as pd
import numpy as np
import re
import string
import itertools
import collections
from collections import OrderedDict, Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

### Load dataset

In [2]:
#df = pd.read_csv('data/datat.csv')
df = pd.read_excel('data/labeled6.xlsx')

#Menampilkan informasi dari jumlah kolom dan baris, serta lima sampel teratas dari dataframe df_items
print('shape : {}'.format(df.shape))
df.head()

shape : (800, 7)


,Nama dan Tanggal,Nama,Tanggal,Rating,Review,Aspect Category,Polarity
0,Hendy Ucok Manurung\n22 April 2021\n\t,Hendy Ucok Manurung,22 April 2021,3,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Ratna Djauhari\n23 April 2021\n\t,Ratna Djauhari,23 April 2021,5,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,Karina Ismir\n22 April 2021\n\t,Karina Ismir,22 April 2021,1,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Sudi Ono\n23 April 2021\n\t,Sudi Ono,23 April 2021,5,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Tetep Herman\n21 April 2021\n\t,Tetep Herman,21 April 2021,1,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Hapus kolom yang tidak akan digunakan

In [3]:
#Mengeliminasi kolom tidak guna
raw = df.drop(columns=['Nama dan Tanggal', 'Nama', 'Tanggal', 'Rating'])
raw.head()

,Review,Aspect Category,Polarity
0,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Cek nilai NaN atau data kosong dengan nilai NaN

In [4]:
print('Jumlah NaN value pada Review : ', len(raw[raw.Review.isna() == True]))

Jumlah NaN value pada Review :  0


### Mengubah nama kolom

In [5]:
raw = raw.rename(columns={'Review':'review','Aspect Category':'aspect_category','Polarity':'polarity'})
raw

,review,aspect_category,polarity
0,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif
...,...,...,...
795,"Semudah diklik, praktis dan sangat membantu👌",Anekdot/Serbaneka,Positif
796,Pelayanannya ramah . Paham dengan bidangnya,Anekdot/Serbaneka,Positif
797,bagus aplikasi nya terasa punya dokter pribadi...,Anekdot/Serbaneka,Positif
798,sangat memudahkan konsultasi tanpa keluar rumah,Pelayanan,Positif


### Save data

In [6]:
raw.to_csv('data/data/data_raw.csv', index=False)

## Text Preprocessing

In [7]:
raw = pd.read_csv('data/data/data_raw.csv')
raw

,review,aspect_category,polarity
0,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif
...,...,...,...
795,"Semudah diklik, praktis dan sangat membantu👌",Anekdot/Serbaneka,Positif
796,Pelayanannya ramah . Paham dengan bidangnya,Anekdot/Serbaneka,Positif
797,bagus aplikasi nya terasa punya dokter pribadi...,Anekdot/Serbaneka,Positif
798,sangat memudahkan konsultasi tanpa keluar rumah,Pelayanan,Positif


### Import kamus data

In [8]:
stop_words = np.array(pd.read_csv("data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)
neg_words = np.array(pd.read_csv("data/external/negative_keyword_ID.txt",
                        sep="\n", header=None).values)
pos_words = np.array(pd.read_csv("data/external/positive_keyword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
root_words = np.array(pd.read_csv("data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
emoticon = pd.read_csv("data/external/emoticon.txt",
                        sep="\t", header=None)
booster_words = np.array(pd.read_csv("data/external/boosterword_ID.txt",
                        sep="\n", header=None).values)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]

In [9]:
slang_words = pd.concat([slang_words, slang, baku_words])
sentiment_words = np.concatenate((pos_words, neg_words, booster_words))
slang_words.drop_duplicates(inplace=True)
emoticon.drop_duplicates(inplace=True)
emoticon = dict(zip(emoticon[0], emoticon[1]))
slang_words = dict(zip(slang_words[0],slang_words[1]))
neg_words = np.unique(neg_words)
pos_words = np.unique(pos_words)
stop_words = np.unique(stop_words)
stop_words = [word for word in stop_words if word not in sentiment_words]

### Jumlah dari kamus data

In [10]:
print("------Dictionary Info---------")
print("Slang words = "  + str(len(slang_words)) + " entries")
print("Emoticon = "  + str(len(emoticon)) + " entries" )
print("Root words = "  + str(len(root_words)) + " entries")
print("Sentiment words = "  + str(len(sentiment_words)) + " entries")

------Dictionary Info---------
Slang words = 2682 entries
Emoticon = 173 entries
Root words = 27979 entries
Sentiment words = 8449 entries


### Fungsi praproses teks

In [11]:
# satuan = ['', 'satu', 'dua', 'tiga', 'empat', 'lima', 'enam', 'tujuh',
#           'delapan', 'sembilan', 'sepuluh', 'sebelas']
    
# def terbilang_(n):
#     if n >= 0 and n <= 11:
#         hasil = [satuan[n]]
#     elif n >= 12 and n <= 19:
#         hasil = terbilang_(n % 10) + ['belas']
#     elif n >= 20 and n <= 99:
#         hasil = terbilang_(n // 10) + ['puluh'] + terbilang_(n % 10)
#     elif n >= 100 and n <= 199:
#         hasil = ['seratus'] + terbilang_(n - 100)
#     elif n >= 200 and n <= 999:
#         hasil = terbilang_(n // 100) + ['ratus'] + terbilang_(n % 100)
#     elif n >= 1000 and n <= 1999:
#         hasil = ['seribu'] + terbilang_(n - 1000)
#     elif n >= 2000 and n <= 999999:
#         hasil = terbilang_(n // 1000) + ['ribu'] + terbilang_(n % 1000)
#     elif n >= 1000000 and n <= 999999999:
#         hasil = terbilang_(n // 1000000) + ['juta'] + terbilang_(n % 1000000)
#     else:
#         hasil = terbilang_(n // 1000000000) + ['milyar'] + terbilang_(n % 100000000)
#     return hasil

# def terbilang(n):
#     if n == 0:
#         return 'nol'
#     t = terbilang_(n)
#     while '' in t:
#         t.remove('')
#     return ' '.join(t)

# def translate_angka(review):
#     number = re.findall("[0-9]+",review)
#     for digit in number:
#         review = re.sub(digit,terbilang(int(digit))+" ",review)
#     return review

def translate_repeating_words(review):
    repeating_words = re.findall(r'\w*(?:2|")',review)
    for word in repeating_words:
        cleaned = word[:-1]
        review = re.sub(word,cleaned + " " + cleaned, review)
    return review

#def delete_suffix_nya(review):
    #return re.sub("(?:nya|ny)[$|\s]"," ",review)

def translate_emoticon(t):
    for w, v in emoticon.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern," " + v + " ",t)
    return t

def translate_non_alpha_num(t):
    non_alpha_num = {
        '%' : 'persen',
        '$' : 'dolar',
        '@' : 'di',
        '&' : 'dan',
        '/' : 'atau',
        '+' : 'plus'
    }
    for w, v in non_alpha_num.items():
        pattern = re.compile(re.escape(w))
        match = re.search(pattern,t)
        if match:
            t = re.sub(pattern,v + " ",t)
    return t

def remove_non_alphanumeric(review):
    return re.sub("[^a-zA-Z\d]"," ", review)

def normalizing_words(review):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(review))

def remove_numeric(review):
    return re.sub("\d", " ", review)

def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]

def remove_stop_words(word_list):
    return [word for word in word_list if word not in stop_words]

def csv_string_to_list(csv_string):
    return csv_string[1:-1].split(",")

def string_without_quotes(word_list):
    new = []
    for word in word_list:
        new.append(word.replace("'",""))
    return new

def convert_list_to_string(word_list):
    return ",".join(word_list)

def remove_single_alphabet_only(review):
    return [word for word in review if word not in string.ascii_lowercase]

def remove_too_short_words(review):
    return [word for word in review if len(word) > 2]

def hapus_katadouble(review):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", review)

#kamus data convert negation
negationWord = ['ga', 'ngga', 'tidak', 
               'bkn', 'tida', 'tak', 
               'jangan', 'enggan', 'gak']

#praproses convert negation
def convert_negation(text):
    words = text.split()

    for index, word in enumerate(words):
        for negation in range(len(negationWord)):
            if words[index] == negationWord[negation]:
                nxt = index + 1 
                words[index] = negationWord[negation] + words[nxt]
                words.pop(nxt)

    return ' '.join(words)

In [12]:
raw_clean = raw

### Praproses Filtering - Mengubah kata mengulang seperti sangat2 -> sangat sangat

In [13]:
raw_clean['review'] = raw_clean['review'].apply(translate_repeating_words)
raw_clean.head()

,review,aspect_category,polarity
0,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Translasi Emotikon

In [14]:
raw_clean['review'] = raw_clean['review'].apply(translate_emoticon)
raw_clean.head()

,review,aspect_category,polarity
0,Sangat Membantu. Terutama di kondisi Pandemi. ...,Pelayanan,Negatif
1,Baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,Terimakasih Halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,Ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Casefoding

In [15]:
raw_clean['review'] = raw_clean['review'].apply(lambda x: x.lower())
raw_clean.head()

,review,aspect_category,polarity
0,sangat membantu. terutama di kondisi pandemi. ...,Pelayanan,Negatif
1,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Filtering - Mengubah simbol non-alpha numerik menjadi string

In [16]:
raw_clean['review'] = raw_clean['review'].apply(translate_non_alpha_num)
raw_clean['review'] = raw_clean['review'].apply(remove_non_alphanumeric)
raw_clean.head()

,review,aspect_category,polarity
0,sangat membantu terutama di kondisi pandemi ...,Pelayanan,Negatif
1,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Filtering - Menghapus kata double

In [17]:
raw_clean['review'] = raw_clean['review'].apply(hapus_katadouble)
raw_clean.head()

,review,aspect_category,polarity
0,sangat membantu terutama di kondisi pandemi ...,Pelayanan,Negatif
1,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,terimakasih halodoc atas pelayanannya sangat b...,Pelayanan,Positif
4,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Filtering - Normalisasi kalimat

In [18]:
raw_clean['review'] = raw_clean['review'].apply(normalizing_words)
raw_clean.head()

,review,aspect_category,polarity
0,sangat membantu terutama di kondisi pandemi ta...,Pelayanan,Negatif
1,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,terimakasih halodoc atas pelayananya sangat ba...,Pelayanan,Positif
4,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Filtering - Menghapus numerik

In [19]:
raw_clean['review'] = raw_clean['review'].apply(remove_numeric)
raw_clean.head()

,review,aspect_category,polarity
0,sangat membantu terutama di kondisi pandemi ta...,Pelayanan,Negatif
1,baru pertama konsultasi dokter lewat halo doc ...,Pelayanan,Negatif
2,astaga obatnya semakin lama beli obat di halo ...,Harga,Negatif
3,terimakasih halodoc atas pelayananya sangat ba...,Pelayanan,Positif
4,ada yang nelpon ke no saya katanya dari halodo...,Sistem,Positif


### Praproses Tokenizing dan Filtering Slang words

In [20]:
raw_clean['review'] = raw_clean['review'].apply(lambda x: x.split())
raw_clean['review'] = raw_clean['review'].apply(lambda x: np.array(x))
raw_clean['review'] = raw_clean['review'].apply(mapping_slang_words)
raw_clean.head()

,review,aspect_category,polarity
0,"[sangat, membantu, terutama, di, kondisi, pand...",Pelayanan,Negatif
1,"[baru, pertama, konsultasi, dokter, lewat, hal...",Pelayanan,Negatif
2,"[astaga, obatnya, semakin, lama, beli, obat, d...",Harga,Negatif
3,"[terima kasih, halodoc, atas, pelayananya, san...",Pelayanan,Positif
4,"[ada, yang, menelepon, ke, nomor, saya, katany...",Sistem,Positif


### Praproses Stopword removal

In [21]:
raw_clean['review'] = raw_clean['review'].apply(remove_stop_words)
raw_clean.head()

,review,aspect_category,polarity
0,"[sangat, membantu, terutama, kondisi, pandemi,...",Pelayanan,Negatif
1,"[konsultasi, dokter, bagus, hanya, durasinya, ...",Pelayanan,Negatif
2,"[astaga, obatnya, lama, beli, obat, harganya, ...",Harga,Negatif
3,"[terima kasih, pelayananya, sangat, baik, cepa...",Pelayanan,Positif
4,"[menelepon, nomor, mendapatkan, hadiah, konfir...",Sistem,Positif


### Casefold kolom sentimen dan aspek

In [22]:
raw_clean['polarity'] =raw_clean['polarity'].apply(lambda x: x.lower())
raw_clean['aspect_category'] =raw_clean['aspect_category'].apply(lambda x: x.lower())
raw_clean.head()

,review,aspect_category,polarity
0,"[sangat, membantu, terutama, kondisi, pandemi,...",pelayanan,negatif
1,"[konsultasi, dokter, bagus, hanya, durasinya, ...",pelayanan,negatif
2,"[astaga, obatnya, lama, beli, obat, harganya, ...",harga,negatif
3,"[terima kasih, pelayananya, sangat, baik, cepa...",pelayanan,positif
4,"[menelepon, nomor, mendapatkan, hadiah, konfir...",sistem,positif


### Save data praproses

In [23]:
raw_clean.to_csv('data/data/data_clean.csv', index=False)

In [24]:
raw = pd.read_csv( "data/data/data_clean.csv")
raw.head()

,review,aspect_category,polarity
0,"['sangat', 'membantu', 'terutama', 'kondisi', ...",pelayanan,negatif
1,"['konsultasi', 'dokter', 'bagus', 'hanya', 'du...",pelayanan,negatif
2,"['astaga', 'obatnya', 'lama', 'beli', 'obat', ...",harga,negatif
3,"['terima kasih', 'pelayananya', 'sangat', 'bai...",pelayanan,positif
4,"['menelepon', 'nomor', 'mendapatkan', 'hadiah'...",sistem,positif


In [25]:
raw['review'] = raw['review'].apply(remove_numeric)
raw.head()

,review,aspect_category,polarity
0,"['sangat', 'membantu', 'terutama', 'kondisi', ...",pelayanan,negatif
1,"['konsultasi', 'dokter', 'bagus', 'hanya', 'du...",pelayanan,negatif
2,"['astaga', 'obatnya', 'lama', 'beli', 'obat', ...",harga,negatif
3,"['terima kasih', 'pelayananya', 'sangat', 'bai...",pelayanan,positif
4,"['menelepon', 'nomor', 'mendapatkan', 'hadiah'...",sistem,positif


In [26]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   review           800 non-null    object
 1   aspect_category  800 non-null    object
 2   polarity         800 non-null    object
dtypes: object(3)
memory usage: 18.9+ KB
None


In [27]:
raw['review'] = raw['review'].apply(csv_string_to_list)
raw

,review,aspect_category,polarity
0,"['sangat', 'membantu', 'terutama', 'kondisi...",pelayanan,negatif
1,"['konsultasi', 'dokter', 'bagus', 'hanya', ...",pelayanan,negatif
2,"['astaga', 'obatnya', 'lama', 'beli', 'oba...",harga,negatif
3,"['terima kasih', 'pelayananya', 'sangat', '...",pelayanan,positif
4,"['menelepon', 'nomor', 'mendapatkan', 'hadi...",sistem,positif
...,...,...,...
795,"['semudah', 'diklik', 'praktis', 'sangat', ...",anekdot/serbaneka,positif
796,"['pelayananya', 'ramah', 'paham', 'bidangnya']",anekdot/serbaneka,positif
797,"['bagus', 'aplikasi', 'dokter', 'pribadi', ...",anekdot/serbaneka,positif
798,"['sangat', 'memudahkan', 'konsultasi', 'tan...",pelayanan,positif


In [28]:
raw['review'] = raw['review'].apply(mapping_slang_words)
raw['review'] = raw['review'].apply(remove_single_alphabet_only)
raw['review'] = raw['review'].apply(remove_too_short_words)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
raw['review']

0      'sangat', 'membantu', 'terutama', 'kondisi', '...
1      'konsultasi', 'dokter', 'bagus', 'hanya', 'dur...
2      'astaga', 'obatnya', 'lama', 'beli', 'obat', '...
3      'terima kasih', 'pelayananya', 'sangat', 'baik...
4      'menelepon', 'nomor', 'mendapatkan', 'hadiah',...
                             ...                        
795    'semudah', 'diklik', 'praktis', 'sangat', 'mem...
796         'pelayananya', 'ramah', 'paham', 'bidangnya'
797    'bagus', 'aplikasi', 'dokter', 'pribadi', 'ter...
798    'sangat', 'memudahkan', 'konsultasi', 'tanpa',...
799    'sangat', 'membantu', 'pelayanan', 'dokter', '...
Name: review, Length: 800, dtype: object

In [29]:
raw.to_csv("data/data/data_clean_clean.csv", index=False)

In [30]:
raw = pd.read_csv( "data/data/data_clean_clean.csv")
raw.head()

,review,aspect_category,polarity
0,"'sangat', 'membantu', 'terutama', 'kondisi', '...",pelayanan,negatif
1,"'konsultasi', 'dokter', 'bagus', 'hanya', 'dur...",pelayanan,negatif
2,"'astaga', 'obatnya', 'lama', 'beli', 'obat', '...",harga,negatif
3,"'terima kasih', 'pelayananya', 'sangat', 'baik...",pelayanan,positif
4,"'menelepon', 'nomor', 'mendapatkan', 'hadiah',...",sistem,positif


In [31]:
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
raw['review'] = raw['review'].apply(lambda x: x.split())
raw['review'] = raw['review'].apply(mapping_slang_words)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
raw['review'] = raw['review'].astype(str)
raw['review'] = raw['review'].apply(lambda x: x.strip())
raw

,review,aspect_category,polarity
0,"'sangat','membantu','terutama','kondisi','pand...",pelayanan,negatif
1,"'konsultasi','dokter','bagus','hanya','durasin...",pelayanan,negatif
2,"'astaga','obatnya','lama','beli','obat','harga...",harga,negatif
3,"'terima,kasih','pelayananya','sangat','baik','...",pelayanan,positif
4,"'menelepon','nomor','mendapatkan','hadiah','ko...",sistem,positif
...,...,...,...
795,"'semudah','diklik','praktis','sangat','membant...",anekdot/serbaneka,positif
796,"'pelayananya','ramah','paham','bidangnya'",anekdot/serbaneka,positif
797,"'bagus','aplikasi','dokter','pribadi','terima,...",anekdot/serbaneka,positif
798,"'sangat','memudahkan','konsultasi','tanpa','ke...",pelayanan,positif


In [32]:
raw.to_csv("data/data/1.2_data_cleaner_pure.csv", index=False)

In [33]:
raw = pd.read_csv("data/data/1.2_data_cleaner_pure.csv")
raw.head()

,review,aspect_category,polarity
0,"'sangat','membantu','terutama','kondisi','pand...",pelayanan,negatif
1,"'konsultasi','dokter','bagus','hanya','durasin...",pelayanan,negatif
2,"'astaga','obatnya','lama','beli','obat','harga...",harga,negatif
3,"'terima,kasih','pelayananya','sangat','baik','...",pelayanan,positif
4,"'menelepon','nomor','mendapatkan','hadiah','ko...",sistem,positif


In [34]:
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
raw

,review,aspect_category,polarity
0,'sangat' 'membantu' 'terutama' 'kondisi' 'pand...,pelayanan,negatif
1,'konsultasi' 'dokter' 'bagus' 'hanya' 'durasin...,pelayanan,negatif
2,'astaga' 'obatnya' 'lama' 'beli' 'obat' 'harga...,harga,negatif
3,'terima kasih' 'pelayananya' 'sangat' 'baik' '...,pelayanan,positif
4,'menelepon' 'nomor' 'mendapatkan' 'hadiah' 'ko...,sistem,positif
...,...,...,...
795,'semudah' 'diklik' 'praktis' 'sangat' 'membant...,anekdot/serbaneka,positif
796,'pelayananya' 'ramah' 'paham' 'bidangnya',anekdot/serbaneka,positif
797,'bagus' 'aplikasi' 'dokter' 'pribadi' 'terima ...,anekdot/serbaneka,positif
798,'sangat' 'memudahkan' 'konsultasi' 'tanpa' 'ke...,pelayanan,positif


In [35]:
raw['review'] = raw['review'].apply(convert_negation)
raw

,review,aspect_category,polarity
0,'sangat' 'membantu' 'terutama' 'kondisi' 'pand...,pelayanan,negatif
1,'konsultasi' 'dokter' 'bagus' 'hanya' 'durasin...,pelayanan,negatif
2,'astaga' 'obatnya' 'lama' 'beli' 'obat' 'harga...,harga,negatif
3,'terima kasih' 'pelayananya' 'sangat' 'baik' '...,pelayanan,positif
4,'menelepon' 'nomor' 'mendapatkan' 'hadiah' 'ko...,sistem,positif
...,...,...,...
795,'semudah' 'diklik' 'praktis' 'sangat' 'membant...,anekdot/serbaneka,positif
796,'pelayananya' 'ramah' 'paham' 'bidangnya',anekdot/serbaneka,positif
797,'bagus' 'aplikasi' 'dokter' 'pribadi' 'terima ...,anekdot/serbaneka,positif
798,'sangat' 'memudahkan' 'konsultasi' 'tanpa' 'ke...,pelayanan,positif


### Praproses Stemming dengan Sastrawi

In [36]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
raw['review'] = raw['review'].apply(lambda x: stemmer.stem(x).replace(" ",","))
raw

,review,aspect_category,polarity
0,"sangat,bantu,utama,kondisi,pandemi,tolong,eval...",pelayanan,negatif
1,"konsultasi,dokter,bagus,hanya,durasi,terlalu,c...",pelayanan,negatif
2,"astaga,obat,lama,beli,obat,harga,tidak,masuk,a...",harga,negatif
3,"terima,kasih,pelayananya,sangat,baik,cepat,sek...",pelayanan,positif
4,"telepon,nomor,dapat,hadiah,konfirmasi,benar,ap...",sistem,positif
...,...,...,...
795,"mudah,klik,praktis,sangat,bantu,bagus",anekdot/serbaneka,positif
796,"pelayananya,ramah,paham,bidang",anekdot/serbaneka,positif
797,"bagus,aplikasi,dokter,pribadi,terima,kasih",anekdot/serbaneka,positif
798,"sangat,mudah,konsultasi,tanpa,keluar,rumah",pelayanan,positif


In [37]:
raw['review'] = raw['review'].apply(lambda x: x.replace("'",""))
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
raw

,review,aspect_category,polarity
0,sangat bantu utama kondisi pandemi tolong eval...,pelayanan,negatif
1,konsultasi dokter bagus hanya durasi terlalu c...,pelayanan,negatif
2,astaga obat lama beli obat harga tidak masuk a...,harga,negatif
3,terima kasih pelayananya sangat baik cepat sek...,pelayanan,positif
4,telepon nomor dapat hadiah konfirmasi benar ap...,sistem,positif
...,...,...,...
795,mudah klik praktis sangat bantu bagus,anekdot/serbaneka,positif
796,pelayananya ramah paham bidang,anekdot/serbaneka,positif
797,bagus aplikasi dokter pribadi terima kasih,anekdot/serbaneka,positif
798,sangat mudah konsultasi tanpa keluar rumah,pelayanan,positif


In [38]:
raw.to_csv('data/data/1.4_data_clean_stemmed.csv',index=False)

In [39]:
raw['review'] = raw['review'].apply(lambda x: x.split())
raw['review'] = raw['review'].apply(mapping_slang_words)
raw['review'] = raw['review'].apply(remove_stop_words)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw

,review,aspect_category,polarity
0,"sangat,bantu,utama,kondisi,pandemi,tolong,eval...",pelayanan,negatif
1,"konsultasi,dokter,bagus,hanya,durasi,terlalu,c...",pelayanan,negatif
2,"astaga,obat,lama,beli,obat,harga,tidak,masuk,a...",harga,negatif
3,"terima,kasih,pelayananya,sangat,baik,cepat,sek...",pelayanan,positif
4,"telepon,nomor,hadiah,konfirmasi,benar,aplikasi...",sistem,positif
...,...,...,...
795,"mudah,klik,praktis,sangat,bantu,bagus",anekdot/serbaneka,positif
796,"pelayananya,ramah,paham,bidang",anekdot/serbaneka,positif
797,"bagus,aplikasi,dokter,pribadi,terima,kasih",anekdot/serbaneka,positif
798,"sangat,mudah,konsultasi,tanpa,keluar,rumah",pelayanan,positif


In [40]:
raw['review'] = raw['review'].apply(lambda x: x.replace("'",""))
raw

,review,aspect_category,polarity
0,"sangat,bantu,utama,kondisi,pandemi,tolong,eval...",pelayanan,negatif
1,"konsultasi,dokter,bagus,hanya,durasi,terlalu,c...",pelayanan,negatif
2,"astaga,obat,lama,beli,obat,harga,tidak,masuk,a...",harga,negatif
3,"terima,kasih,pelayananya,sangat,baik,cepat,sek...",pelayanan,positif
4,"telepon,nomor,hadiah,konfirmasi,benar,aplikasi...",sistem,positif
...,...,...,...
795,"mudah,klik,praktis,sangat,bantu,bagus",anekdot/serbaneka,positif
796,"pelayananya,ramah,paham,bidang",anekdot/serbaneka,positif
797,"bagus,aplikasi,dokter,pribadi,terima,kasih",anekdot/serbaneka,positif
798,"sangat,mudah,konsultasi,tanpa,keluar,rumah",pelayanan,positif


In [41]:
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
raw

,review,aspect_category,polarity
0,sangat bantu utama kondisi pandemi tolong eval...,pelayanan,negatif
1,konsultasi dokter bagus hanya durasi terlalu c...,pelayanan,negatif
2,astaga obat lama beli obat harga tidak masuk a...,harga,negatif
3,terima kasih pelayananya sangat baik cepat sek...,pelayanan,positif
4,telepon nomor hadiah konfirmasi benar aplikasi...,sistem,positif
...,...,...,...
795,mudah klik praktis sangat bantu bagus,anekdot/serbaneka,positif
796,pelayananya ramah paham bidang,anekdot/serbaneka,positif
797,bagus aplikasi dokter pribadi terima kasih,anekdot/serbaneka,positif
798,sangat mudah konsultasi tanpa keluar rumah,pelayanan,positif


In [42]:
raw['review'] = raw['review'].apply(hapus_katadouble)
raw

,review,aspect_category,polarity
0,sangat bantu utama kondisi pandemi tolong eval...,pelayanan,negatif
1,konsultasi dokter bagus hanya durasi terlalu c...,pelayanan,negatif
2,astaga obat lama beli obat harga tidak masuk a...,harga,negatif
3,terima kasih pelayananya sangat baik cepat sek...,pelayanan,positif
4,telepon nomor hadiah konfirmasi benar aplikasi...,sistem,positif
...,...,...,...
795,mudah klik praktis sangat bantu bagus,anekdot/serbaneka,positif
796,pelayananya ramah paham bidang,anekdot/serbaneka,positif
797,bagus aplikasi dokter pribadi terima kasih,anekdot/serbaneka,positif
798,sangat mudah konsultasi tanpa keluar rumah,pelayanan,positif


In [43]:
raw.to_csv('data/data/1.4_data_clean_stemmed_2.csv',index=False)

In [44]:
import pickle
import imblearn
from sklearn import model_selection, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, log_loss,confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [45]:
data = pd.read_csv('data/data/1.4_data_clean_stemmed_2.csv')
data

,review,aspect_category,polarity
0,sangat bantu utama kondisi pandemi tolong eval...,pelayanan,negatif
1,konsultasi dokter bagus hanya durasi terlalu c...,pelayanan,negatif
2,astaga obat lama beli obat harga tidak masuk a...,harga,negatif
3,terima kasih pelayananya sangat baik cepat sek...,pelayanan,positif
4,telepon nomor hadiah konfirmasi benar aplikasi...,sistem,positif
...,...,...,...
795,mudah klik praktis sangat bantu bagus,anekdot/serbaneka,positif
796,pelayananya ramah paham bidang,anekdot/serbaneka,positif
797,bagus aplikasi dokter pribadi terima kasih,anekdot/serbaneka,positif
798,sangat mudah konsultasi tanpa keluar rumah,pelayanan,positif


In [46]:
data['aspek_sentimen'] = data['aspect_category'] +" "+ data['polarity'] 

In [47]:
data

,review,aspect_category,polarity,aspek_sentimen
0,sangat bantu utama kondisi pandemi tolong eval...,pelayanan,negatif,pelayanan negatif
1,konsultasi dokter bagus hanya durasi terlalu c...,pelayanan,negatif,pelayanan negatif
2,astaga obat lama beli obat harga tidak masuk a...,harga,negatif,harga negatif
3,terima kasih pelayananya sangat baik cepat sek...,pelayanan,positif,pelayanan positif
4,telepon nomor hadiah konfirmasi benar aplikasi...,sistem,positif,sistem positif
...,...,...,...,...
795,mudah klik praktis sangat bantu bagus,anekdot/serbaneka,positif,anekdot/serbaneka positif
796,pelayananya ramah paham bidang,anekdot/serbaneka,positif,anekdot/serbaneka positif
797,bagus aplikasi dokter pribadi terima kasih,anekdot/serbaneka,positif,anekdot/serbaneka positif
798,sangat mudah konsultasi tanpa keluar rumah,pelayanan,positif,pelayanan positif


In [48]:
data.groupby('aspect_category').size().sort_values(ascending=False)

aspect_category
pelayanan            396
anekdot/serbaneka    302
sistem                83
harga                 19
dtype: int64

In [49]:
data.groupby('polarity').size().sort_values(ascending=False)

polarity
positif    650
negatif    150
dtype: int64

In [50]:
catagories_2 = data['aspek_sentimen'].unique()
catagories_2 , len(catagories_2)

(array(['pelayanan negatif', 'harga negatif', 'pelayanan positif',
        'sistem positif', 'harga positif', 'sistem negatif',
        'anekdot/serbaneka positif', 'anekdot/serbaneka negatif'],
       dtype=object),
 8)

In [51]:
# Memisahkan data menjadi data latih dan data uji
# untuk keperluan pembangunan model
from sklearn.model_selection import train_test_split
X = data['review']
y = data['aspek_sentimen']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [52]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(640,) (640,)
(160,) (160,)


In [53]:
# Proses pembobotan atau vektorisasi
print("Creating tfidf..")
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize
vectorizer = TfidfVectorizer(analyzer = "word",
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = None, 
                             max_features = 5000) 

# Melakukan pengubahan data latih kedalam vektor fitur
%time train_data_features = vectorizer.fit_transform(X_train)

# Mengkonversi dari list ke array, untuk memudahkan 
# ke tahap klasifikasi
train_data_features = train_data_features.toarray()

Creating tfidf..
Wall time: 606 ms


In [54]:
# Sama seperti pada data latih, data tes diubah kedalam
# vektor fitur selanjutnya di konversi menjadi bentuk array
test_data_features = vectorizer.transform(X_test)
test_data_features = test_data_features.toarray()

In [55]:
print(train_data_features.shape)
print(test_data_features.shape)

(640, 902)
(160, 902)


In [56]:
# Inisialisasi ETC
etc = ExtraTreesClassifier(criterion='gini',class_weight='balanced_subsample', max_features=10)

# Memasang data latih ke classifier
etc.fit(train_data_features,y_train)

# Melakukan prediksi dengan membandingkan dengan data tes
y_pred_train = etc.predict(train_data_features)
y_pred_cat = etc.predict(test_data_features)

In [57]:
pickle.dump(y_pred_cat, open('pred_cat', 'wb'))

In [58]:
# comparing actual response values (y_train) with predicted response values (y_pred_train) 
metrics.accuracy_score(y_train, y_pred_train)*100

98.28125

In [59]:
# comparing actual response values (y_test) with predicted response values (y_pred_cat) 
metrics.accuracy_score(y_test, y_pred_cat)*100

58.12500000000001

In [60]:
print(y_train.shape, y_pred_train.shape)
print(y_test.shape, y_pred_cat.shape)

(640,) (640,)
(160,) (160,)


In [61]:
print(classification_report(y_test, y_pred_cat ,target_names=catagories_2))

                           precision    recall  f1-score   support

        pelayanan negatif       0.33      0.20      0.25         5
            harga negatif       0.60      0.57      0.58        60
        pelayanan positif       0.00      0.00      0.00         2
           sistem positif       0.00      0.00      0.00         2
            harga positif       0.62      0.36      0.45        14
           sistem negatif       0.60      0.77      0.67        60
anekdot/serbaneka positif       0.55      0.60      0.57        10
anekdot/serbaneka negatif       0.25      0.14      0.18         7

                 accuracy                           0.58       160
                macro avg       0.37      0.33      0.34       160
             weighted avg       0.56      0.58      0.56       160



C:\Users\dell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
# Melakukan proses pengujian
acc = accuracy_score(y_test, y_pred_cat)
cm = confusion_matrix(y_test, y_pred_cat)
print("Accuracy Score: " + str(acc))
print("Confusion Matrix: "+ str(cm))
print("Precision Score: "+ str(precision_score(y_test,y_pred_cat,average='macro')))
print("Recall Score: "+ str(recall_score(y_test,y_pred_cat,average='macro')))
print("F1 Score: "+ str(f1_score(y_test,y_pred_cat,average='macro')))

Accuracy Score: 0.58125
Confusion Matrix: [[ 1  2  0  0  0  1  1  0]
 [ 0 34  0  0  0 22  1  3]
 [ 0  1  0  0  1  0  0  0]
 [ 0  0  0  0  0  2  0  0]
 [ 2  2  0  0  5  2  3  0]
 [ 0 13  0  0  1 46  0  0]
 [ 0  2  0  0  1  1  6  0]
 [ 0  3  0  0  0  3  0  1]]
Precision Score: 0.3684602130325815
Recall Score: 0.32916666666666666
F1 Score: 0.33881520446301466


C:\Users\dell\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
y_test.to_csv('data/data_test_aspek_1.csv',index=False)

In [64]:
# Menguji model prediksi yang telah di buat
# dengan data masukan yang baru
tes = ['Membantu pelayanannya terimakasih, dokternya keren puas deh dan cepat respon ramah',
'Bagus banget pelayanannya, cepat respon dokternya dan ramah sangat kebantu',
'Terimakasih dokter telah melayani istri saya yang hamil',
'Suka banget sama aplikasinya, mudah cepat dipakai terimakasih keren',
'Sangat membantu, bagus aplikasinya mudah dipakai terimakasih',
'Daftar vaksinasi di aplikasi terus kemaren saya ikut vaksinasi langsung dapet sertifikatnya terimakasih',
'Sumpah alternatif banget pas miris uang, hemat pakai aplikasi cocok untuk anak kosan harganya murah suka deh',
'Bisa konsultasi di rumah dengan harga yang sangat murah dan bikin hemat',
'Sangat membantu suka',
'Sangat membantu terimakasih',
'Bagus terimakasih telah membantu',
'balita telinga anak belah kanan cairan dan bau dan dikasih obat langsung sehat terimakasih dokter',
'Harga obat lumayan murah dan hemat terimakasih',
'Membantu pelayanannya terimakasih',
'Membantu pelayanannya terimakasih',
'Membantu pelayanannya terimakasih',
'dokternya baik',
'dokternya sabar, ramah bagus deh',
'terimakasih dokter saya senang konsultasi',
'puas deh konsultasi di halodoc cepat, jelas dan bagus pelayanannya',
'obatnya bagus langsung sembuh',
'terimakasih sudah dapat obatnya jadi sehat, terimakasih dokter baik dan pintar',
'bagus konsultasi di halodoc pintar2 dokternya dan ramah',
'aplikasi membantu cepat bagus',
'aplikasi membantu cepat bagus',
'aplikasi membantu cepat bagus',
'aplikasinya lengkap lancar',
'terimakasih membantu aplikasinya',
'aplikasi pintar',
'aplikasi yang membantu kesehatan',
'bagus banget lengkap pilihan di aplikasi terimakasih',
'bagus aplikasinya',
'puas pakai aplikasi ini',
'terimakasih konsultasi murah',
'terimakasih konsultasi murah',
'terimakasih obat murah bagus',
'terimakasih obat murah jadi hemat untuk anak kosan',
'cocok untuk anak kosan hemat, murah',
'obatnya murah',
'harganya murah bagus deh',
'bagus bikin hemat uang',
'harganya pas untuk obat2 murah',
'murah deh pasti puas',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'sangat membantu',
'puas konsultasinya',
'senang suka dokter2 nya bagus',
'Chatnya di batalin mulu udah mahal lagi pelayanannya tolong',
'Respon CSnya lambat sekali jelek halodoc tolong bantuannya',
'Konsultasi sama dokter bagus tapi durasinya terlalu cepat banget bikin jelek trus repotin tolong bantuannya',
'Jujur puas banget nasihatnya jelas dokternya, sayangkan konsultasi di aplikasinya otomatis berhenti terus kadang di batalin kok jadi jelek sih',
'Log out mulu terus verifikasi kode malah ngirim sms yang error bikin hp lambat aja tolong',
'Aplikasi freeze mulu bikin hp lambat tolong bantuannya bikin repot aja',
'Aplikasinya lengkap, jelas dan cepat kerjanya tapi kok suka nge freeze mulu',
'Parah masa konsultasi doang bayar jelek halodoc mahal2',
'Tolong bantuannya kok konsultasi bayar sih di iklan mah gratis terus katanya bagus pada murah jelek ah',
'Dokternya bagus2 untuk konsultasi tapi kenapa mahal banget pas bayar sih',
'janjian mau rapid tapi malah di batalin tolong bantuannya, katanya bagus tapi malah jelek',
'coba kaji lagi vaksin untuk mitra ojol sejati mau cari rezeki malah bikin repot di batalin tolong bantuannya admin malah jadi jelek gini halodoc',
'pas pertama kali pakai aplikasi halodoc bagus banget pada baik dokternya dan respon cepat tapi sekarang malah jadi lambat respon, aplikasinya error mulu dan di batalin mulu sama dokternya tolong bantuannya',
'Chatnya jelek',
'Chatnya jelek ga di respon',
'Chatnya jelek dokternya batalin mulu',
'Chatnya jelek ga ada respon banget tolong lah',
'CS NYA GA DI RESPON KEMANA LAMBAT BANGET DEH JELEK ANJIR',
'Pelayanan yang buruk maaf jadi males kesini',
'Obat datengnya lambat banget jadi sakit nih',
'Obat kosong mulu jadi males pesen deh jelek ih',
'error mulu anjir',
'halooo aplikasi jelek apa kabar',
'jelek tidak di sarankan untuk yg lain error mulu',
'bikin hp lambat dan freeze mulu aplikasinya',
'aplikasi jelek gimanasih',
'sering banget di rekomen apotek yang kosong gimana sih tolong bantuannya',
'di kasih apotek yang kosong wkwkwkwk goblok',
'tolong error mulu di hp saya',
'mahal banget',
'mahal banget',
'mahal banget lah jadi boros uang gw',
'jelek mahal konsultasi aja',
'bayar terus woi gratis kek jelek amat',
'harga tidak konsisten mahal mulu',
'mahal di aplikasi ini mah',
'wkwkwk jelek banget berbayar semuanya',
'jelek banget pas anak saya sakit tidak membantu',
'jelek aplikasinya',
'jelek aplikasinya',
'jelek tidak suka sama app ini',
'daftar vaksinasi tapi tiba2 di batalin',
'mau vaksin gagal mulu tolong',
'tolong kemaren saya di saranin obat A tapi malah dapet obat B',
'tidak di saranin untuk pake aplikasi ini jelek banget']
# tes_features = vectorizer.transform(tes).toarray()

est = vectorizer.transform(tes)
test_aspect = pd.DataFrame(est.toarray(), columns=vectorizer.get_feature_names())
test_aspect.to_csv("data/data/fitur_test_aspect.csv")
# print(tes_features)
print(etc.predict(test_aspect))

['pelayanan positif' 'pelayanan positif' 'pelayanan positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'harga positif' 'pelayanan positif'
 'sistem positif' 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'pelayanan positif' 'harga positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'pelayanan positif' 'pelayanan positif' 'anekdot/serbaneka positif'
 'pelayanan positif' 'pelayanan positif' 'pelayanan positif'
 'pelayanan positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'anekdot/serbaneka positif'
 'anekdot/serbaneka positif' 'harga positif' 'harga positif'
 'harga positif' 'harga positif' 'harga positif' 'harga positif'
 'harga positif' 'pelayanan positif' 'harga positif' 'harga positif'
 'anekdot/serbaneka